# Gold Layer: CO2 Emissions National Dashboard

## Purpose
Extract and transform CO2 emissions data from fact_economic_indicators (1980-2024) into business-ready national emissions tracking dashboard.

## Data Sources
- **Input**: final-spark-silver/fact_economic_indicators (Environmental category)
- **Dimensions**: dim_time, dim_location, dim_indicator
- **Output**: gold_co2_emissions_summary, gold_co2_trends_visualization

In [1]:
# Install delta-spark if not already installed
import subprocess
import sys

try:
    import delta
    print("Delta Spark already installed")
except ImportError:
    print("Installing Delta Spark...")
    subprocess.check_call([sys.executable, "-m", "pip", "install", "delta-spark==2.4.0"])
    print("Delta Spark installation completed")

Delta Spark already installed


In [2]:
# Import required libraries
import os
from datetime import datetime

from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.window import Window

print("All libraries imported successfully")

All libraries imported successfully


In [3]:
# Initialize Spark session with Delta Lake configuration
def create_spark_session():
    try:
        # Full Delta Lake configuration
        spark = SparkSession.builder \
            .appName("GoldCO2Emissions") \
            .config("spark.jars.packages", "io.delta:delta-core_2.12:2.4.0") \
            .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
            .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
            .config("spark.serializer", "org.apache.spark.serializer.KryoSerializer") \
            .config("spark.sql.adaptive.enabled", "true") \
            .config("spark.sql.adaptive.coalescePartitions.enabled", "true") \
            .config("spark.databricks.delta.retentionDurationCheck.enabled", "false") \
            .config("spark.databricks.delta.schema.autoMerge.enabled", "true") \
            .getOrCreate()
        
        # Test Delta Lake functionality
        from delta.tables import DeltaTable
        print("Delta Lake successfully configured")
        return spark, True
        
    except Exception as e:
        print(f"Delta Lake configuration failed: {e}")
        print("Falling back to basic Spark session...")
        
        # Fallback: Basic Spark session
        spark = SparkSession.builder \
            .appName("GoldCO2Emissions") \
            .config("spark.serializer", "org.apache.spark.serializer.KryoSerializer") \
            .config("spark.sql.adaptive.enabled", "true") \
            .config("spark.sql.adaptive.coalescePartitions.enabled", "true") \
            .getOrCreate()
        
        return spark, False

# Create Spark session
spark, delta_available = create_spark_session()

# Set logging level
spark.sparkContext.setLogLevel("WARN")

print(f"Spark session initialized: {spark.version}")
print(f"Delta Lake available: {delta_available}")
print(f"Processing timestamp: {datetime.now()}")

your 131072x1 screen size is bogus. expect trouble
25/08/19 13:06:28 WARN Utils: Your hostname, 3rnese resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
25/08/19 13:06:28 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/home/ernese/miniconda3/envs/SO/lib/python3.10/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/ernese/.ivy2/cache
The jars for the packages stored in: /home/ernese/.ivy2/jars
io.delta#delta-core_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-972d7927-628f-4f84-b849-fd15f261a0d7;1.0
	confs: [default]
	found io.delta#delta-core_2.12;2.4.0 in central
	found io.delta#delta-storage;2.4.0 in central
	found org.antlr#antlr4-runtime;4.9.3 in central
:: resolution report :: resolve 146ms :: artifacts dl 8ms
	:: modules in use:
	io.delta#delta-core_2.12;2.4.0 from central in [default]
	io.delta#delta-storage;2.4.0 from central in [default]
	org.antlr#antlr4-runtime;4.9.3 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   3   |   0   |

Delta Lake successfully configured
Spark session initialized: 3.4.0
Delta Lake available: True
Processing timestamp: 2025-08-19 13:06:30.813273


In [4]:
# Define paths
SILVER_PATH = "/home/ernese/miniconda3/envs/SO/New_SO/final-spark-silver"
GOLD_PATH = "/home/ernese/miniconda3/envs/SO/New_SO/final-spark-gold"

# Create gold directory if it doesn't exist
os.makedirs(GOLD_PATH, exist_ok=True)

print(f"Silver layer path: {SILVER_PATH}")
print(f"Gold layer path: {GOLD_PATH}")
print(f"Delta Lake support: {delta_available}")

Silver layer path: /home/ernese/miniconda3/envs/SO/New_SO/final-spark-silver
Gold layer path: /home/ernese/miniconda3/envs/SO/New_SO/final-spark-gold
Delta Lake support: True


## Step 1: Load Source Data

In [5]:
# Smart data loading function
def load_table(table_name, silver_path):
    table_path = f"{silver_path}/{table_name}"
    
    if not os.path.exists(table_path):
        raise FileNotFoundError(f"Table {table_name} not found at {table_path}")
    
    # Check if it's a Delta table
    delta_log_path = os.path.join(table_path, "_delta_log")
    
    if os.path.exists(delta_log_path) and delta_available:
        try:
            df = spark.read.format("delta").load(table_path)
            print(f"Loaded {table_name} as Delta format")
            return df
        except Exception as e:
            print(f"Failed to load {table_name} as Delta: {e}")
    
    # Fallback to Parquet
    try:
        df = spark.read.parquet(table_path)
        print(f"Loaded {table_name} as Parquet format")
        return df
    except Exception as e:
        print(f"Failed to load {table_name} as Parquet: {e}")
        raise

# Load silver layer tables
try:
    fact_economic_indicators = load_table("fact_economic_indicators", SILVER_PATH)
    dim_time = load_table("dim_time", SILVER_PATH)
    dim_location = load_table("dim_location", SILVER_PATH)
    dim_indicator = load_table("dim_indicator", SILVER_PATH)
    
    print("\nSuccessfully loaded all silver layer tables")
    
except Exception as e:
    print(f"Error loading silver tables: {str(e)}")
    raise

Loaded fact_economic_indicators as Delta format
Loaded dim_time as Delta format
Loaded dim_location as Delta format
Loaded dim_indicator as Delta format

Successfully loaded all silver layer tables


In [6]:
# Display data validation and explore CO2-related data
print(f"Data Validation:")
print(f"fact_economic_indicators records: {fact_economic_indicators.count():,}")
print(f"dim_time records: {dim_time.count():,}")
print(f"dim_location records: {dim_location.count():,}")
print(f"dim_indicator records: {dim_indicator.count():,}")

# Filter for CO2 and emissions-related data
co2_data = fact_economic_indicators.filter(
    (col("category") == "Environmental") | 
    (lower(col("category")).contains("co2")) |
    (lower(col("category")).contains("emission"))
)

print(f"\nCO2/Emissions related records: {co2_data.count():,}")

print("\nAvailable Categories:")
categories = fact_economic_indicators.select("category").distinct().orderBy("category")
categories.show()

print("\nDate Range in CO2 Data:")
if co2_data.count() > 0:
    date_range = co2_data.select(
        min("year").alias("min_year"),
        max("year").alias("max_year"),
        countDistinct("year").alias("total_years")
    ).collect()[0]
    
    print(f"Years covered: {date_range['min_year']} - {date_range['max_year']} ({date_range['total_years']} years)")
else:
    print("No CO2/emissions data found in Environmental category")
    # Show sample data to understand structure
    print("\nSample fact_economic_indicators data:")
    fact_economic_indicators.show(10, truncate=False)

print("\nSample CO2/Environmental Data:")
co2_data.show(5, truncate=False)

Data Validation:


25/08/19 13:06:35 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


fact_economic_indicators records: 364
dim_time records: 612
dim_location records: 34
dim_indicator records: 15

CO2/Emissions related records: 111

Available Categories:
+-------------+
|     category|
+-------------+
|     Economic|
|Environmental|
+-------------+


Date Range in CO2 Data:
Years covered: 1980 - 2023 (44 years)

Sample CO2/Environmental Data:
+---------------------+-----------+-------+------------+----------------+----+-----+-------+-----------------------+------------------------------+-----+---------------+-------------------------+-------------------------+---------------+-------------+-----------+---------+---------------+------------------+------------------+-----------+------------------------------------------------------------------------------------------------------------------------+--------------------------+--------------------------+
|economic_indicator_id|location_id|date_id|indicator_id|measurement_date|year|month|quarter|indicator_code         |indicat

## Step 2: Data Preparation and CO2 Data Extraction

In [7]:
# Check if we have CO2 data, if not, filter broader environmental indicators
if co2_data.count() == 0:
    print("No specific CO2 data found, checking all environmental indicators...")
    co2_data = fact_economic_indicators.filter(col("category") == "Environmental")
    
    if co2_data.count() == 0:
        print("No Environmental category found, using all available data for analysis...")
        co2_data = fact_economic_indicators
        print(f"Using all economic indicators: {co2_data.count():,} records")
    else:
        print(f"Using Environmental category: {co2_data.count():,} records")
else:
    print(f"Found specific CO2 data: {co2_data.count():,} records")

# Create base dataset for CO2 analysis
base_co2_data = co2_data.withColumn(
    "quarter", 
    when(col("year").isNotNull(), 
         when(col("year") % 4 == 0, 1)
         .when(col("year") % 4 == 1, 2)
         .when(col("year") % 4 == 2, 3)
         .otherwise(4)
    ).otherwise(1)
).withColumn(
    "fiscal_year",
    col("year")
)

# Join with location dimension if available
if dim_location.count() > 0:
    print("Joining with location dimension...")
    
    location_cols = dim_location.columns
    
    if "region_code" in location_cols:
        co2_with_location = base_co2_data.join(
            dim_location.select("location_id", "location_name", "location_type", "region_code"),
            base_co2_data.location_id == dim_location.location_id,
            "left"
        ).withColumnRenamed("region_code", "region")
    else:
        co2_with_location = base_co2_data.join(
            dim_location.select("location_id", "location_name", "location_type"),
            base_co2_data.location_id == dim_location.location_id,
            "left"
        ).withColumn("region", lit("Philippines"))
else:
    print("No location data available, using default location info")
    co2_with_location = base_co2_data.withColumn(
        "location_name", lit("Philippines")
    ).withColumn(
        "location_type", lit("National")
    ).withColumn(
        "region", lit("National")
    )

print(f"Base CO2 dataset prepared with {co2_with_location.count():,} records")

Found specific CO2 data: 111 records
Joining with location dimension...
Base CO2 dataset prepared with 111 records


## Step 3: Calculate CO2 Trend Metrics

In [8]:
# Calculate rolling averages and trend indicators for CO2 emissions
window_5yr = Window.partitionBy("category").orderBy("year").rowsBetween(-4, 0)
window_prev_year = Window.partitionBy("category").orderBy("year")
window_decade = Window.partitionBy("category").orderBy("year").rowsBetween(-9, 0)

co2_with_trends = co2_with_location.withColumn(
    "rolling_5yr_avg", avg("value").over(window_5yr)
).withColumn(
    "prev_year_value", lag("value", 1).over(window_prev_year)
).withColumn(
    "rolling_10yr_avg", avg("value").over(window_decade)
).withColumn(
    "year_over_year_change", col("value") - col("prev_year_value")
).withColumn(
    "year_over_year_pct_change", 
    when(col("prev_year_value") != 0, 
         (col("value") - col("prev_year_value")) / abs(col("prev_year_value")) * 100
    ).otherwise(0.0)
)

print("Calculated CO2 trend metrics and rolling averages")

Calculated CO2 trend metrics and rolling averages


In [9]:
# Create CO2 emissions classifications and chart formatting
co2_with_classification = co2_with_trends.withColumn(
    "emissions_trend_direction",
    when(col("year_over_year_pct_change") > 5.0, "Rapidly Increasing")
    .when(col("year_over_year_pct_change") > 1.0, "Increasing")
    .when(col("year_over_year_pct_change") < -5.0, "Rapidly Decreasing")
    .when(col("year_over_year_pct_change") < -1.0, "Decreasing")
    .otherwise("Stable")
).withColumn(
    "environmental_impact_category",
    when(col("year_over_year_pct_change") > 3, "High Impact")
    .when(col("year_over_year_pct_change") > 0, "Moderate Impact")
    .when(col("year_over_year_pct_change") < -3, "Improving")
    .otherwise("Stable")
).withColumn(
    "emissions_intensity_level",
    when(col("value") > col("rolling_10yr_avg") * 1.2, "Very High")
    .when(col("value") > col("rolling_10yr_avg") * 1.1, "High")
    .when(col("value") < col("rolling_10yr_avg") * 0.9, "Low")
    .when(col("value") < col("rolling_10yr_avg") * 0.8, "Very Low")
    .otherwise("Normal")
).withColumn(
    "chart_color_hex",
    when(col("emissions_trend_direction") == "Rapidly Increasing", "#8B0000")  # Dark Red
    .when(col("emissions_trend_direction") == "Increasing", "#DC143C")  # Crimson
    .when(col("emissions_trend_direction") == "Stable", "#FFD700")  # Gold
    .when(col("emissions_trend_direction") == "Decreasing", "#32CD32")  # Lime Green
    .when(col("emissions_trend_direction") == "Rapidly Decreasing", "#228B22")  # Forest Green
    .otherwise("#808080")  # Gray
)

print("Added CO2 emissions classifications and chart formatting")

# Show sample of processed CO2 data
print("\nSample Processed CO2 Data:")
co2_with_classification.filter(
    col("year") >= 2015
).select(
    "year", "category", "value", "rolling_5yr_avg", 
    "year_over_year_pct_change", "emissions_trend_direction", "environmental_impact_category"
).orderBy("year", "category").show(10)

Added CO2 emissions classifications and chart formatting

Sample Processed CO2 Data:
+----+-------------+-----+------------------+-------------------------+-------------------------+-----------------------------+
|year|     category|value|   rolling_5yr_avg|year_over_year_pct_change|emissions_trend_direction|environmental_impact_category|
+----+-------------+-----+------------------+-------------------------+-------------------------+-----------------------------+
|2015|Environmental|209.5|119.05999999999999|        372.9119638826185|       Rapidly Increasing|                  High Impact|
|2015|Environmental|113.2|            133.44|       -45.96658711217184|       Rapidly Decreasing|                    Improving|
|2015|Environmental| 48.2|            103.84|       -57.42049469964664|       Rapidly Decreasing|                    Improving|
|2016|Environmental|219.9|127.02000000000001|        356.2240663900414|       Rapidly Increasing|                  High Impact|
|2016|Environmental

## Step 4: Create Gold Table 1 - CO2 Emissions Summary

In [10]:
# Create annual CO2 emissions summary
co2_emissions_summary = co2_with_classification.groupBy(
    "year", "fiscal_year"
).agg(
    # Emissions indicators
    sum("value").alias("total_emissions"),
    avg("value").alias("avg_emissions_indicator"),
    max("value").alias("max_emissions_indicator"),
    min("value").alias("min_emissions_indicator"),
    
    # Per capita and intensity metrics (assuming population data available)
    avg("value").alias("emissions_per_capita_proxy"),  # Simplified proxy
    
    # Growth and change rates
    avg("year_over_year_pct_change").alias("emissions_annual_change_pct"),
    avg("rolling_5yr_avg").alias("emissions_5yr_avg"),
    avg("rolling_10yr_avg").alias("emissions_10yr_avg"),
    
    # Trend indicators
    count(when(col("emissions_trend_direction") == "Increasing", 1)).alias("increasing_indicators_count"),
    count(when(col("emissions_trend_direction") == "Decreasing", 1)).alias("decreasing_indicators_count")
).withColumn(
    "co2_summary_id", monotonically_increasing_id()
).withColumn(
    "emissions_intensity_score",
    # Composite intensity score based on current vs historical levels
    when(col("total_emissions") > 0,
         (col("total_emissions") / (col("emissions_5yr_avg") + 1)) * 100
    ).otherwise(0.0)
).withColumn(
    "environmental_performance_rating",
    when(col("emissions_annual_change_pct") < -5, "Excellent")
    .when(col("emissions_annual_change_pct") < -1, "Good")
    .when(col("emissions_annual_change_pct") < 1, "Fair")
    .when(col("emissions_annual_change_pct") < 5, "Poor")
    .otherwise("Critical")
).withColumn(
    "emissions_trend_overall",
    when(col("decreasing_indicators_count") > col("increasing_indicators_count"), "Improving")
    .when(col("increasing_indicators_count") > col("decreasing_indicators_count"), "Worsening")
    .otherwise("Stable")
).withColumn(
    "climate_action_urgency",
    when(col("emissions_annual_change_pct") > 10, "Critical")
    .when(col("emissions_annual_change_pct") > 5, "High")
    .when(col("emissions_annual_change_pct") > 0, "Moderate")
    .otherwise("Low")
).withColumn(
    "processing_timestamp", current_timestamp()
).withColumn(
    "data_source", lit("World Bank Environmental Data via Silver Layer")
).orderBy("year")

print(f"Created CO2 emissions summary with {co2_emissions_summary.count():,} records")

# Show sample data
print("\nSample CO2 Emissions Summary (Recent Years):")
co2_emissions_summary.filter(col("year") >= 2015).select(
    "year", "total_emissions", "emissions_annual_change_pct", 
    "environmental_performance_rating", "emissions_trend_overall", "climate_action_urgency"
).show(10)

Created CO2 emissions summary with 44 records

Sample CO2 Emissions Summary (Recent Years):
+----+------------------+---------------------------+--------------------------------+-----------------------+----------------------+
|year|   total_emissions|emissions_annual_change_pct|environmental_performance_rating|emissions_trend_overall|climate_action_urgency|
+----+------------------+---------------------------+--------------------------------+-----------------------+----------------------+
|2015|             370.9|          89.84162735693333|                        Critical|                 Stable|              Critical|
|2016|393.59999999999997|          84.86139478772462|                        Critical|                 Stable|              Critical|
|2017|             444.0|          90.99269564862283|                        Critical|                 Stable|              Critical|
|2018|             486.5|          87.91349594569623|                        Critical|                 S

## Step 5: Create Gold Table 2 - CO2 Trends Visualization

In [11]:
# Create chart-optimized CO2 trends table
co2_trends_viz = co2_with_classification.select(
    monotonically_increasing_id().alias("co2_trend_id"),
    col("year"),
    col("quarter"),
    col("fiscal_year"),
    concat(col("year"), lit("-Q"), col("quarter")).alias("chart_period"),
    col("category"),
    col("value").alias("emissions_value"),
    col("rolling_5yr_avg").alias("trend_line_value"),
    col("year_over_year_change"),
    col("year_over_year_pct_change"),
    col("emissions_trend_direction"),
    col("environmental_impact_category"),
    col("emissions_intensity_level"),
    col("chart_color_hex"),
    
    # Chart formatting
    concat(
        lit("Emissions: "), 
        round(col("value"), 2),
        lit(" MT CO2e")
    ).alias("chart_label"),
    
    # Tooltip text for interactive charts
    concat(
        col("year"), lit(": "),
        round(col("value"), 2), lit(" MT CO2e "),
        when(coalesce(col("year_over_year_pct_change"), lit(0)) > 0, 
             concat(lit("(up "), round(col("year_over_year_pct_change"), 1), lit("%)"))
        ).when(coalesce(col("year_over_year_pct_change"), lit(0)) < 0,
             concat(lit("(down "), round(abs(col("year_over_year_pct_change")), 1), lit("%)"))
        ).otherwise(lit("(stable)")),
        lit(" - "), col("emissions_trend_direction")
    ).alias("tooltip_text"),
    
    # Trend arrows for dashboard display
    when(col("emissions_trend_direction") == "Rapidly Increasing", "double_up")
    .when(col("emissions_trend_direction") == "Increasing", "up")
    .when(col("emissions_trend_direction") == "Rapidly Decreasing", "double_down")
    .when(col("emissions_trend_direction") == "Decreasing", "down")
    .otherwise("stable").alias("trend_arrow"),
    
    # Choropleth map values (for future geographic visualization)
    when(col("value") > 0, log10(col("value") + 1)).otherwise(0.0).alias("choropleth_value"),
    
    # Performance indicators for dashboard
    when(col("environmental_impact_category") == "High Impact", "critical")
    .when(col("environmental_impact_category") == "Moderate Impact", "warning")
    .when(col("environmental_impact_category") == "Improving", "success")
    .otherwise("neutral").alias("performance_status"),
    
    # Target tracking (Philippines NDC targets)
    lit(75.0).alias("ndc_target_reduction_pct"),  # Philippines NDC target
    when(col("year_over_year_pct_change") < -2, "on_track")
    .when(col("year_over_year_pct_change") > 2, "off_track")
    .otherwise("monitoring").alias("target_achievement_status"),
    
    col("location_name"),
    current_timestamp().alias("last_updated")
).orderBy("year", "category")

print(f"Created CO2 trends visualization table with {co2_trends_viz.count():,} records")

# Show sample visualization data
print("\nSample CO2 Trends Visualization (2015-2024):")
co2_trends_viz.filter(
    col("year") >= 2015
).select(
    "year", "emissions_value", "trend_line_value", 
    "year_over_year_pct_change", "trend_arrow", "performance_status", "target_achievement_status"
).show(15)

Created CO2 trends visualization table with 111 records

Sample CO2 Trends Visualization (2015-2024):
+----+---------------+------------------+-------------------------+-----------+------------------+-------------------------+
|year|emissions_value|  trend_line_value|year_over_year_pct_change|trend_arrow|performance_status|target_achievement_status|
+----+---------------+------------------+-------------------------+-----------+------------------+-------------------------+
|2015|          209.5|119.05999999999999|        372.9119638826185|  double_up|          critical|                off_track|
|2015|          113.2|            133.44|       -45.96658711217184|double_down|           success|                 on_track|
|2015|           48.2|            103.84|       -57.42049469964664|double_down|           success|                 on_track|
|2016|          219.9|127.02000000000001|        356.2240663900414|  double_up|          critical|                off_track|
|2016|          121.3| 

## Step 6: Data Quality Validation

In [12]:
# Data quality validation
print("Data Quality Validation:")

# Get year range for reporting
year_stats = co2_emissions_summary.agg(
    min("year").alias("min_year"),
    max("year").alias("max_year"),
    count("year").alias("total_records")
).collect()[0]

min_year = year_stats['min_year']
max_year = year_stats['max_year']
total_records = year_stats['total_records']

print(f"Year range: {min_year} - {max_year} ({total_records} records)")

# Check for nulls in critical fields
critical_fields = ["year", "total_emissions", "emissions_annual_change_pct"]
null_checks = {}

for field in critical_fields:
    if field in co2_emissions_summary.columns:
        null_count = co2_emissions_summary.filter(col(field).isNull()).count()
        null_checks[field] = null_count
    else:
        null_checks[field] = "Field not found"

print(f"\nNull values in CO2 emissions summary:")
for field, null_count in null_checks.items():
    print(f"  {field}: {null_count}")

# Environmental performance rating distribution
print(f"\nEnvironmental Performance Rating Distribution:")
if "environmental_performance_rating" in co2_emissions_summary.columns:
    co2_emissions_summary.groupBy("environmental_performance_rating").count().orderBy("environmental_performance_rating").show()
else:
    print("Environmental performance rating field not found")

# Emissions trend analysis
print(f"\nEmissions Trend Analysis:")
if "emissions_trend_overall" in co2_emissions_summary.columns:
    co2_emissions_summary.groupBy("emissions_trend_overall").count().show()
else:
    print("Emissions trend field not found")

# Recent trends summary
print(f"\nRecent CO2 Trends (2015-2024):")
recent_data = co2_emissions_summary.filter(col("year") >= 2015)

if recent_data.count() > 0:
    recent_trends = recent_data.agg(
        avg("total_emissions").alias("avg_emissions"),
        avg("emissions_annual_change_pct").alias("avg_change_rate"),
        avg("emissions_intensity_score").alias("avg_intensity_score")
    ).collect()[0]
    
    for metric, value in recent_trends.asDict().items():
        if value is not None:
            print(f"  {metric}: {value:.2f}")
        else:
            print(f"  {metric}: No data")
else:
    print("No recent data available")

print("\nData quality validation completed")

Data Quality Validation:
Year range: 1980 - 2023 (44 records)

Null values in CO2 emissions summary:
  year: 0
  total_emissions: 0
  emissions_annual_change_pct: 0

Environmental Performance Rating Distribution:
+--------------------------------+-----+
|environmental_performance_rating|count|
+--------------------------------+-----+
|                        Critical|   43|
|                       Excellent|    1|
+--------------------------------+-----+


Emissions Trend Analysis:
+-----------------------+-----+
|emissions_trend_overall|count|
+-----------------------+-----+
|                 Stable|   44|
+-----------------------+-----+


Recent CO2 Trends (2015-2024):
  avg_emissions: 439.60
  avg_change_rate: 73.24
  avg_intensity_score: 290.25

Data quality validation completed


## Step 7: Save Gold Tables as Delta

In [13]:
# Smart save function
def save_gold_table(dataframe, table_name, gold_path, delta_available):
    table_path = f"{gold_path}/{table_name}"
    
    if delta_available:
        try:
            # Save as Delta table
            dataframe.write \
                .format("delta") \
                .mode("overwrite") \
                .save(table_path)
            
            print(f"Successfully saved {table_name} as Delta table to {table_path}")
            print(f"   Records: {dataframe.count():,}")
            return True, "delta"
            
        except Exception as e:
            print(f"Failed to save {table_name} as Delta: {e}")
            print("Falling back to Parquet format...")
    
    # Fallback to Parquet
    try:
        dataframe.write \
            .mode("overwrite") \
            .parquet(table_path + "_parquet")
        
        print(f"Successfully saved {table_name} as Parquet to {table_path}_parquet")
        print(f"   Records: {dataframe.count():,}")
        return True, "parquet"
        
    except Exception as e:
        print(f"Failed to save {table_name}: {e}")
        return False, "failed"

# Save gold tables
print("Saving gold tables...")

# Save CO2 emissions summary
success1, format1 = save_gold_table(
    co2_emissions_summary, 
    "gold_co2_emissions_summary", 
    GOLD_PATH, 
    delta_available
)

# Save CO2 trends visualization
success2, format2 = save_gold_table(
    co2_trends_viz, 
    "gold_co2_trends_visualization", 
    GOLD_PATH, 
    delta_available
)

print(f"\nSave Results:")
print(f"  CO2 Emissions Summary: {'Success' if success1 else 'Failed'} ({format1})")
print(f"  CO2 Trends Visualization: {'Success' if success2 else 'Failed'} ({format2})")

Saving gold tables...
Successfully saved gold_co2_emissions_summary as Delta table to /home/ernese/miniconda3/envs/SO/New_SO/final-spark-gold/gold_co2_emissions_summary
   Records: 44
Successfully saved gold_co2_trends_visualization as Delta table to /home/ernese/miniconda3/envs/SO/New_SO/final-spark-gold/gold_co2_trends_visualization
   Records: 111

Save Results:
  CO2 Emissions Summary: Success (delta)
  CO2 Trends Visualization: Success (delta)


## Step 8: Create Processing Report

In [14]:
# Generate processing report
processing_report = {
    "processing_timestamp": datetime.now().isoformat(),
    "notebook_name": "gold_co2_emissions_national.ipynb",
    "processing_type": "co2_emissions_gold_layer",
    "delta_lake_available": delta_available,
    
    "input_data": {
        "source_table": "fact_economic_indicators (Environmental category)",
        "date_range": f"{min_year}-{max_year}" if min_year and max_year else "Unknown",
        "total_years": max_year - min_year + 1 if min_year and max_year else 0,
        "input_records": co2_data.count()
    },
    
    "output_tables": {
        "gold_co2_emissions_summary": {
            "path": f"{GOLD_PATH}/gold_co2_emissions_summary",
            "records": co2_emissions_summary.count(),
            "format": format1,
            "success": success1,
            "purpose": "National CO2 emissions tracking dashboard"
        },
        "gold_co2_trends_visualization": {
            "path": f"{GOLD_PATH}/gold_co2_trends_visualization",
            "records": co2_trends_viz.count(),
            "format": format2,
            "success": success2,
            "purpose": "Chart-optimized CO2 emissions trends with choropleth map support"
        }
    },
    
    "data_quality": {
        "null_checks": null_checks,
        "year_range_complete": True,
        "environmental_categories": "Environmental data processed"
    },
    
    "climate_features": [
        "NDC target tracking and achievement status",
        "Climate action urgency indicators",
        "Choropleth map values for geographic visualization",
        "Environmental impact categorization",
        "Emissions intensity scoring"
    ],
    
    "business_value": [
        f"{max_year - min_year + 1 if min_year and max_year else 'Multiple'}-year CO2 emissions tracking",
        "Environmental performance rating system",
        "Climate action urgency assessment",
        "Philippines NDC target monitoring",
        "Chart-ready data for choropleth mapping",
        "Policy impact assessment for climate initiatives"
    ],
    
    "status": "completed"
}

# Save report as JSON
import json
report_path = f"{GOLD_PATH}/co2_emissions_processing_report.json"
with open(report_path, 'w') as f:
    json.dump(processing_report, f, indent=2)

print("Processing Report:")
print(json.dumps(processing_report, indent=2))
print(f"\nReport saved to: {report_path}")

Processing Report:
{
  "processing_timestamp": "2025-08-19T13:06:56.123690",
  "notebook_name": "gold_co2_emissions_national.ipynb",
  "processing_type": "co2_emissions_gold_layer",
  "delta_lake_available": true,
  "input_data": {
    "source_table": "fact_economic_indicators (Environmental category)",
    "date_range": "1980-2023",
    "total_years": 44,
    "input_records": 111
  },
  "output_tables": {
    "gold_co2_emissions_summary": {
      "path": "/home/ernese/miniconda3/envs/SO/New_SO/final-spark-gold/gold_co2_emissions_summary",
      "records": 44,
      "format": "delta",
      "success": true,
      "purpose": "National CO2 emissions tracking dashboard"
    },
    "gold_co2_trends_visualization": {
      "path": "/home/ernese/miniconda3/envs/SO/New_SO/final-spark-gold/gold_co2_trends_visualization",
      "records": 111,
      "format": "delta",
      "success": true,
      "purpose": "Chart-optimized CO2 emissions trends with choropleth map support"
    }
  },
  "data_qu

## Step 9: Verification and Sample Queries

In [15]:
# Verification function
def load_gold_table(table_name, gold_path, expected_format):
    if expected_format == "delta":
        try:
            return spark.read.format("delta").load(f"{gold_path}/{table_name}")
        except:
            print(f"Failed to load {table_name} as Delta, trying Parquet...")
    
    # Try Parquet backup
    try:
        return spark.read.parquet(f"{gold_path}/{table_name}_parquet")
    except:
        return spark.read.parquet(f"{gold_path}/{table_name}")

# Verify saved tables
print("Verification - Reading saved gold tables:")

try:
    test_summary = load_gold_table("gold_co2_emissions_summary", GOLD_PATH, format1)
    print(f"gold_co2_emissions_summary: {test_summary.count():,} records")
    
    test_trends = load_gold_table("gold_co2_trends_visualization", GOLD_PATH, format2)
    print(f"gold_co2_trends_visualization: {test_trends.count():,} records")
    
    print("\nSample Dashboard Queries:")
    
    # Query 1: Recent CO2 emissions performance
    print("\n1. Recent CO2 Emissions Performance (Last 8 years):")
    test_summary.filter(col("year") >= max_year - 7).select(
        "year", "total_emissions", "emissions_annual_change_pct", 
        "environmental_performance_rating", "emissions_trend_overall", "climate_action_urgency"
    ).orderBy(desc("year")).show(8)
    
    # Query 2: CO2 trends for choropleth map
    print("\n2. CO2 Trends for Choropleth Mapping (Recent years):")
    choropleth_data = test_trends.filter(
        col("year") >= max_year - 4
    )
    
    if choropleth_data.count() > 0:
        choropleth_data.select(
            "year", "emissions_value", "choropleth_value", 
            "trend_arrow", "performance_status", "target_achievement_status"
        ).orderBy("year").show(10, truncate=False)
    else:
        print("No choropleth data available for recent years")
    
    # Query 3: Climate action urgency distribution
    print(f"\n3. Climate Action Urgency Distribution:")
    test_summary.groupBy("climate_action_urgency").count().orderBy("climate_action_urgency").show()
    
    # Query 4: NDC target achievement status
    print(f"\n4. NDC Target Achievement Status (Recent years):")
    ndc_status = test_trends.filter(col("year") >= 2020)
    if ndc_status.count() > 0:
        ndc_status.groupBy("target_achievement_status").count().show()
    else:
        print("No NDC status data available")

except Exception as e:
    print(f"Error during verification: {str(e)}")

print(f"\nGold layer CO2 emissions national dashboard implementation completed successfully!")
print(f"Tables saved in {format1.upper()} format")
print(f"Ready for choropleth mapping and climate dashboard integration")

Verification - Reading saved gold tables:
gold_co2_emissions_summary: 44 records
gold_co2_trends_visualization: 111 records

Sample Dashboard Queries:

1. Recent CO2 Emissions Performance (Last 8 years):
+----+------------------+---------------------------+--------------------------------+-----------------------+----------------------+
|year|   total_emissions|emissions_annual_change_pct|environmental_performance_rating|emissions_trend_overall|climate_action_urgency|
+----+------------------+---------------------------+--------------------------------+-----------------------+----------------------+
|2023|             244.0|          32.59832659377307|                        Critical|                 Stable|              Critical|
|2022|             517.8|          63.40856506052243|                        Critical|                 Stable|              Critical|
|2021|             508.4|          67.07235634978177|                        Critical|                 Stable|              Cr

In [16]:
# Stop Spark session
spark.stop()
print("Spark session stopped")

Spark session stopped
